In [49]:
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [50]:
standings_url ="https://fbref.com/en/comps/9/Premier-League-Stats"

In [51]:
import time
yearRange = list(range(2025, 2015, -1))
every_match = [] # each element is a dataframe for a specific team to make
                  # a bigger dataframe of all the teams

#Get every season link for our time range
for year in yearRange:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    #First column of the table has links to every team
    standings_table = soup.select('table.stats_table')[0]
    time.sleep(7)
    indivTeamLink = [l.get("href") for l in standings_table.find_all('a')]
    indivTeamLink = [l for l in indivTeamLink if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in indivTeamLink]


    #Setup link to go to the previous season
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
      # Get the team name
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
      # Get all the match fixture information for that team for that specific season
      # Total of 38 games
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        indivTeamLink = [l.get("href") for l in soup.find_all('a')]
      # Get shooting stats for the current team for each matchs
        indivTeamLink = [l for l in indivTeamLink if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{indivTeamLink[0]}")
        time.sleep(7)
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
      # Merge the two dataframes together for match fixture information and shooting data
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist","PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
      # Append the individual dataframe of the team to the list of dfs
        team_data["Season"] = year
        team_data["Team"] = team_name
        every_match.append(team_data)
        time.sleep(7)

<ipython-input-51-b0af550ff63b>:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
<ipython-input-51-b0af550ff63b>:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
<ipython-input-51-b0af550ff63b>:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
<ipython-input-51-b0af550ff63b>:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO

In [54]:
match_df = pd.concat(every_match)

In [55]:
match_df.columns = [c.lower() for c in match_df.columns]
#lower case all column titles

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,Anthony Taylor,Match Report,NaN,11.0,5.0,19.1,0,0,2025,Manchester City
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,Samuel Allison,Match Report,NaN,13.0,4.0,17.8,1,1,2025,Manchester City
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,Michael Oliver,Match Report,NaN,23.0,8.0,15.0,0,0,2025,Manchester City
4,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,W,2,1,Brentford,...,Darren Bond,Match Report,NaN,18.0,7.0,17.2,0,0,2025,Manchester City
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,Tim Robinson,Match Report,NaN,18.0,5.0,14.8,0,0,2025,Liverpool


In [56]:
from google.colab import files
match_df = match_df.sort_values('date')
match_df.to_csv("combined_matches_2025-25_to_2015-16.csv")

In [57]:
download = files.download('combined_matches_2025-25_to_2015-16.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>